<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-07 11:22:57
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20


-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: 30.01 K (0.22%)
Current PnL: -13.10 L (-9.13%)
CY Booked + Current PnL: -6.14 L (-4.28%)
-------------------
Total profit:  4.66 L
Total loss:  -17.76 L
-------------------
Total Booked + Current PnL: 20.84 L (17.88%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.08%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.55 L (52.22%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.94%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,6904.43,8285.00,-16.52,H-LC,92.71,113798.0,10232.0,10481.0,0.44,9.88,9.21,20.00,32.0,0.98,0.83,25.34,X40N,BTT,HEALTHCARE
88,UNIONBANK,123.87,163.00,0.15,M-LC,31.25,174370.0,33530.0,10968.0,0.18,23.81,6.29,31.59,88.0,3.06,1.27,51.47,XY24,NTT,BANKS
39,HINDALCO,651.95,761.55,-19.90,H-LC,87.50,110728.0,6416.0,11117.0,-1.06,6.15,10.04,16.81,27.0,0.58,0.81,23.18,X5K,ATH,METALS
71,SBIN,760.30,863.00,-11.51,M-LC,32.29,215140.0,13660.0,13554.0,0.59,6.78,6.30,13.51,77.0,1.01,1.57,20.23,XY25,NTT,BANKS
92,VIPIND,488.80,489.00,-211.61,H-SC,97.92,80467.0,-14849.0,14886.0,-1.40,-15.58,18.50,0.04,157.0,-1.00,0.59,58.71,OX40N,NTT,MISC


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,HEROMOTOCO,4311.81,6039.03,-5.71,H-MC,37.5,151378.0,465.0,59991.0,-0.46,0.31,39.63,40.06,103.0,0.01,1.11,23.82,AR,ATH,AUTO
60,NATIONALUM,189.63,247.44,-50.15,H-MC,50.0,100271.0,-233.0,30873.0,-1.51,-0.23,30.79,30.49,97.0,-0.01,0.73,32.10,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-18.87,L-SC,100.00,163122.0,21280.0,35283.0,13.67,15.00,21.63,39.88,262.0,0.60,1.19,65.31,XY24,NTT,POWER
65,RELAXO,902.64,1176.00,-27.37,H-SC,95.83,65626.0,-49912.0,84900.0,10.32,-43.20,129.37,30.28,131.0,-0.59,0.48,28.21,X40N,NTT,FOOTWEAR
3,ALKYLAMINE,2347.98,4546.37,11.63,X-SC,98.96,103944.0,2981.0,91554.0,5.32,2.95,88.08,93.63,67.0,0.03,0.76,54.82,SR,ATH,CHEMICALS
26,DABUR,505.20,735.00,-7.36,X-MC,84.38,200499.0,3471.0,86154.0,3.84,1.76,42.97,45.49,56.0,0.04,1.46,13.89,XY24,BTT,FMCG
27,DEN,52.79,75.00,122.80,L-SC,85.42,62752.0,-21712.0,57249.0,3.78,-25.71,91.23,42.07,261.0,-0.38,0.46,33.27,AR,NTT,ENTERTAINMENT


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,SURYODAY,189.78,240.0,53.51,H-SC,7.29,108609.0,-41317.0,80990.0,-2.61,-27.56,74.57,26.46,147.0,-0.51,0.79,39.14,XR,NTT,BANKS
70,SATIN,199.77,274.0,-15.90,H-SC,72.92,165298.0,-30676.0,103493.0,-2.42,-15.65,62.61,37.16,135.0,-0.30,1.21,23.44,XY24,NTT,FINANCE
87,UJJIVANSFB,52.77,53.0,48.19,M-SC,28.65,128115.0,-14364.0,14989.0,-2.29,-10.08,11.70,0.44,247.0,-0.96,0.94,51.21,OX40N,NTT,BANKS
33,GREENPANEL,375.16,537.0,211.37,M-SC,73.96,139056.0,-44022.0,122995.0,-1.93,-24.05,88.45,43.14,217.0,-0.36,1.02,30.80,XY24,NTT,MISC
24,COFFEEDAY,59.14,80.0,-55.73,L-SC,80.21,66278.0,-47271.0,87321.0,-1.76,-41.63,131.75,35.27,265.0,-0.54,0.48,62.22,XR,NTT,HOTELS


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,48.19,M-SC,28.65,128115.0,-14364.0,14989.0,-2.29,-10.08,11.70,0.44,247.0,-0.96,0.94,51.21,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-29.18,M-SC,28.65,150991.0,-20150.0,20097.0,1.94,-11.77,13.31,-0.03,191.0,-1.00,1.10,8.52,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,94.79,149023.0,-25536.0,25572.0,0.19,-14.63,17.16,0.02,133.0,-1.00,1.09,29.23,OX40N,NTT,PAINTS
92,VIPIND,488.80,489.0,-211.61,H-SC,97.92,80467.0,-14849.0,14886.0,-1.40,-15.58,18.50,0.04,157.0,-1.00,0.59,58.71,OX40N,NTT,MISC
81,TATAELXSI,7332.28,7332.0,-17.65,X-MC,8.33,86429.0,-16223.0,16223.0,-0.57,-15.80,18.77,-0.00,59.0,-1.00,0.63,31.38,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-18.06,X-LC,40.62,266682.0,-29252.0,123154.0,-0.02,-9.88,46.18,31.73,1.0,-0.24,1.95,6.74,X40,BTT,IT
48,INFY,1461.46,2275.00,-8.83,X-LC,76.04,292325.0,30724.0,114913.0,-0.46,11.74,39.31,55.67,2.0,0.27,2.13,18.57,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-14.11,X-LC,55.21,269046.0,-1301.0,35918.0,2.69,-0.48,13.35,12.81,5.0,-0.04,1.96,12.52,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,16.86,X-LC,62.50,267766.0,38533.0,28758.0,0.86,16.81,10.74,29.35,10.0,1.34,1.96,27.23,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-2.64,X-LC,26.04,82352.0,-27820.0,116890.0,0.05,-25.25,141.94,80.85,11.0,-0.24,0.60,21.29,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.9,NaN,NaN,90.62,73845.0,39396.0,554.0,2.07,114.36,0.75,115.97,274.0,71.11,0.54,12.92,AR,ATH,ELECTRICAL
88,UNIONBANK,123.87,163.0,0.15,M-LC,31.25,174370.0,33530.0,10968.0,0.18,23.81,6.29,31.59,88.0,3.06,1.27,51.47,XY24,NTT,BANKS


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,91.10,M-MC,38.54,229284.0,4322.0,163342.0,-1.17,1.92,71.24,74.53,189.0,0.03,1.67,33.12,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.64,27.50,H-SC,47.92,130624.0,5327.0,53752.0,0.19,4.25,41.15,47.15,143.0,0.10,0.95,63.42,MH,ATH,POWER
20,CAMS,3643.00,5250.99,3.87,H-SC,27.08,116760.0,14756.0,30264.0,-1.46,14.47,25.92,44.14,126.0,0.49,0.85,33.06,X40N,ATH,MISC
88,UNIONBANK,123.87,163.00,0.15,M-LC,31.25,174370.0,33530.0,10968.0,0.18,23.81,6.29,31.59,88.0,3.06,1.27,51.47,XY24,NTT,BANKS
15,BLUESTARCO,1646.70,2337.55,-2.82,H-SC,96.88,182290.0,17620.0,51460.0,-0.99,10.70,28.23,41.95,124.0,0.34,1.33,18.96,X40N,ATH,AC


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,90.62,73845.0,39396.0,554.0,2.07,114.36,0.75,115.97,274.0,71.11,0.54,12.92,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.43,M-SC,61.46,203169.0,31548.0,130516.0,0.32,18.38,64.24,94.43,201.0,0.24,1.48,49.95,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.86,M-LC,89.58,166718.0,15773.0,93679.0,-0.43,10.45,56.19,72.51,71.0,0.17,1.22,14.84,XR,NTT,IT
54,KPIGREEN,497.21,731.64,27.50,H-SC,47.92,130624.0,5327.0,53752.0,0.19,4.25,41.15,47.15,143.0,0.10,0.95,63.42,MH,ATH,POWER
45,INDIAMART,2327.09,4850.92,-48.45,H-SC,65.62,134355.0,11019.0,122747.0,-0.46,8.93,91.36,108.45,123.0,0.09,0.98,33.28,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,22.92,17919.0,-31674.0,598.0,2.57,-63.87,3.34,-62.66,270.0,-52.97,0.13,20.16,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,80.21,66278.0,-47271.0,87321.0,-1.76,-41.63,131.75,35.27,265.0,-0.54,0.48,62.22,XR,NTT,HOTELS
58,MASFIN,326.60,399.50,-19.15,H-SC,57.81,92130.0,-5850.0,27722.0,-1.02,-5.97,30.09,22.32,145.0,-0.21,0.67,33.62,XR,ATH,FINANCE
9,BAJAJFINSV,1994.80,2758.00,-1.80,X-LC,35.42,98034.0,289.0,37106.0,0.32,0.30,37.85,38.26,18.0,0.01,0.72,30.90,X40,BTT,FINANCE
3,ALKYLAMINE,2347.98,4546.37,11.63,X-SC,98.96,103944.0,2981.0,91554.0,5.32,2.95,88.08,93.63,67.0,0.03,0.76,54.82,SR,ATH,CHEMICALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-18.87,L-SC,100.00,163122.0,21280.0,35283.0,13.67,15.00,21.63,39.88,262.0,0.60,1.19,65.31,XY24,NTT,POWER
3,ALKYLAMINE,2347.98,4546.37,11.63,X-SC,98.96,103944.0,2981.0,91554.0,5.32,2.95,88.08,93.63,67.0,0.03,0.76,54.82,SR,ATH,CHEMICALS
92,VIPIND,488.80,489.00,-211.61,H-SC,97.92,80467.0,-14849.0,14886.0,-1.40,-15.58,18.50,0.04,157.0,-1.00,0.59,58.71,OX40N,NTT,MISC
15,BLUESTARCO,1646.70,2337.55,-2.82,H-SC,96.88,182290.0,17620.0,51460.0,-0.99,10.70,28.23,41.95,124.0,0.34,1.33,18.96,X40N,ATH,AC
65,RELAXO,902.64,1176.00,-27.37,H-SC,95.83,65626.0,-49912.0,84900.0,10.32,-43.20,129.37,30.28,131.0,-0.59,0.48,28.21,X40N,NTT,FOOTWEAR


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.91
1,25,41.87
2,50,70.53


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.48
LC    33.92
MC    21.55
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.82
X40      12.98
XY25     12.02
X40N     11.93
XR       10.07
OX40N     8.20
AR        8.18
MH        1.68
X5K       1.45
X200      1.40
SR        1.25
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.55
M-SC    15.91
H-LC    14.09
X-LC    12.91
H-MC     9.48
X-MC     7.66
M-LC     5.88
X-SC     4.42
M-MC     4.05
L-SC     2.60
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.20,-0.50,34.11
IT,12.10,-9.75,63.67
FINANCE,8.53,-11.60,52.09
BANKS,7.79,-6.97,50.61
MISC,5.82,-20.72,76.89
PAINTS,5.70,-11.91,33.97
ELECTRICAL,5.49,2.11,43.60
HEALTHCARE,4.59,-2.45,32.37
AUTO,4.31,-14.39,64.70


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2669661.0,26
XR,1080648.0,13
AR,1024574.0,10
X40,676998.0,10
X40N,527324.0,11
XY25,473257.0,8
OX40N,306986.0,11
SR,173407.0,2
MH,84625.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1969529.0,20
M-SC,1764936.0,21
H-LC,608790.0,12
X-LC,542375.0,8
H-MC,536176.0,8
M-MC,415751.0,3
X-MC,401660.0,6
X-SC,314125.0,5
M-LC,255046.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      881513.0      6
M-SC       XY24      809603.0      7
H-SC       AR        524316.0      3
M-SC       XR        334714.0      4
M-MC       XY24      306423.0      2
H-MC       XY24      299782.0      3
X-LC       X40       275173.0      3
H-SC       XR        217388.0      3
           X40N      205269.0      4
H-LC       X40N      194601.0      4
M-SC       AR        183166.0      2
           OX40N     162565.0      5
H-LC       X40       153846.0      3
L-SC       XR        148921.0      2
M-LC       XY24      147813.0      2
X-MC       X40       146468.0      2
X-SC       XR        123138.0      1
M-SC       XY25      121564.0      1
X-LC       AR        116890.0      1
M-MC       XR        109328.0      1
X-LC       XY25      105372.0      3
X-MC       XY25      100206.0      1
M-LC       XR         93679.0      1
X-SC       SR         91554.0      1
H-SC       OX40N      87291.0      3
X-MC       XY24       86154.0      1
H-MC       XY25       85585.0      1
H-LC       AR         82408.0      1
M-SC       SR         81853.0      1
           X40        71471.0      1
H-LC       X5K        69003.0      2
           X200       68290.0      1
H-MC       AR         59991.0      1
X-SC       XY24       58526.0      1
L-SC       AR         57249.0      1
H-SC       MH         53752.0      1
L-MC       XR         53480.0      1
X-MC       X40N       52609.0      1
L-LC       XY25       46976.0      1
X-LC       X40N       44940.0      1
X-SC       OX40N      40907.0      2
H-LC       XY24       40642.0      1
L-SC       XY24       35283.0      1
H-MC       MH         30873.0      1
           X40        30040.0      1
           X40N       29905.0      1
X-MC       OX40N      16223.0      1
M-LC       XY25       13554.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 47.0 seconds
